In [ ]:
# System Installation / Preparation (macOS Homebrew)
brew install graphviz

In [ ]:
# Pip Installation / Preparation
pip install pydot
pip install tensorflow
pip install scikit-learn

In [ ]:
# Python imports
import tensorflow as tf
import keras
from keras import layers
import pydot
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Load data as pandas dataframe
df = pd.read_csv("./data.csv")

# Print: first records
df.head()

In [ ]:
# Get dimension of input data frame
num_rows, num_cols = df.shape
print(num_rows, num_cols)

In [ ]:
# Select the record sets from the input data frame
record_sets = []
set_start_index = 0
set_end_index = 2
for index in range(int(num_cols/2)):
    record_sets.append(df.iloc[0:,set_start_index:set_end_index])
    set_start_index = set_start_index + 2
    set_end_index = set_end_index + 2

# Print: first record set
record_sets[0]

In [ ]:
# Check the winner for each record in all record sets
wins = []
for index in range(len(record_sets)):
    winner_arr = []
    record_set = record_sets[index]
    for row in range(num_rows):
        #print(record_set.iloc[row,0], record_set.iloc[row,1])
        if record_set.iloc[row,1] == record_set.iloc[row,0]:
            winner_arr.append(0)                                # tie
        elif record_set.iloc[row,1] == 'R':                     # player = record_set[row,1]
            if record_set.iloc[row,0] == 'P':                   # computer = record_set[row,0]
                winner_arr.append(-1)                           # player lose
            if record_set.iloc[row,0] == 'S':
                winner_arr.append(1)                            # player win
        elif record_set.iloc[row,1] == 'S':
            if record_set.iloc[row,0] == 'R':
                winner_arr.append(-1)
            if record_set.iloc[row,0] == 'P':
                winner_arr.append(1)
        elif record_set.iloc[row,1] == 'P':
            if record_set.iloc[row,0] == 'S':
                winner_arr.append(-1)
            if record_set.iloc[row,0] == 'R':
                winner_arr.append(1)
    wins.append(winner_arr)

# Print: wins array of arrays ( = 2D array)
print(wins)

In [ ]:
# Convert wins array to Pandas data frame
win_dfs = []
for index in range(len(wins)):
    header = 'W' + str(index + 1)
    win_df = pd.DataFrame(wins[index], columns=[header])
    win_dfs.append(win_df)

# Print: first data frame
win_dfs[0]

In [ ]:
# Create OneHotEncoder categories of the input data frame
enc = OneHotEncoder()
enc.fit(df.to_numpy().flatten().reshape(-1, 1))

# Print: array of OneHotEncoder categories
enc.categories_

In [ ]:
# Sample one hot encoding by the created OneHotEncoder
sample = enc.transform(np.array('R').reshape(-1, 1))
sample.toarray()

In [ ]:
# One hot encoding of the input data frame
one_hot_df = df.applymap(lambda x: enc.transform(np.array(x).reshape(-1, 1)).toarray()[0])

# Print: one hot encoded input data frame
one_hot_df

In [ ]:
# Select the one hot encoded record sets from the input data frame
one_hot_record_sets = []
one_hot_set_start_index = 0
one_hot_set_end_index = 2
for index in range(int(num_cols/2)):
    one_hot_record_sets.append(one_hot_df.iloc[0:,one_hot_set_start_index:one_hot_set_end_index])
    one_hot_set_start_index = one_hot_set_start_index + 2
    one_hot_set_end_index = one_hot_set_end_index + 2

# Print: first one hot encoded record set
one_hot_record_sets[0]

In [ ]:
# Combine/Concat one hot-encoded record sets with the wins data frames
combined_record_sets = []
for index in range(len(one_hot_record_sets)):
    stacked = pd.concat([one_hot_record_sets[index], win_dfs[index]], axis=1)
    combined_record_sets.append(stacked)

# Print: first combined record set
combined_record_sets[0]

In [ ]:
# Set the number of rows for the training and validation data
training_rows = round(num_rows * 0.7)
print(training_rows)

In [ ]:
# Split combined record sets into training and validation data
training_record_sets = []
validation_record_sets = []
for index in range(len(combined_record_sets)):
    record_set = combined_record_sets[index]
    # split
    training_set = record_set.iloc[:training_rows]
    validation_set = record_set.iloc[training_rows:]
    # append
    training_record_sets.append(training_set)     
    validation_record_sets.append(validation_set)       

# Print: first training record set
training_record_sets[0]

In [ ]:
# Rename headers for merging
renamed_training_sets = []
renamed_validation_sets = []
# Training data sets
for index in range(len(training_record_sets)):
    record_set = training_record_sets[index]
    header_com = 'C' + str(index + 1)
    header_human = 'H' + str(index + 1)
    header_win = 'W' + str(index + 1)
    renamed_training_sets.append(record_set.rename(columns={header_com: "C", header_human: "H", header_win: "W"}))
# Validation data sets
for index in range(len(validation_record_sets)):
    record_set = validation_record_sets[index]
    header_com = 'C' + str(index + 1)
    header_human = 'H' + str(index + 1)
    header_win = 'W' + str(index + 1)
    renamed_validation_sets.append(record_set.rename(columns={header_com: "C", header_human: "H", header_win: "W"}))

# Print: first renamed validation sets
renamed_validation_sets[0]

In [ ]:

# Merge training and valitation record sets into one training and one validation data frame
# Create training and validation data drame
x_training_df = pd.DataFrame(columns=['C', 'H', 'W'])
y_training_df = pd.DataFrame(columns=['C', 'H', 'W'])

x_validation_df = pd.DataFrame(columns=['C', 'H', 'W'])
y_validation_df = pd.DataFrame(columns=['C', 'H', 'W'])
# Training
for index in range(len(renamed_training_sets)):
    training_set = renamed_training_sets[index]
    validation_set = renamed_validation_sets[index]
    # x
    x_training_df = pd.concat([x_training_df, training_set], axis=0)
    # y
    y_training_df = pd.concat([y_training_df, training_set.iloc[1:,0:]], axis=0)
    y_training_df = pd.concat([y_training_df, validation_set.iloc[0:1,0:]], axis=0)
# Valitation
for index in range(len(renamed_validation_sets)):
    validation_set = renamed_validation_sets[index]
    # x
    x_validation_df = pd.concat([x_validation_df, validation_set.iloc[:-1,0:]], axis=0)
    # y
    y_validation_df = pd.concat([y_validation_df, validation_set.iloc[1:,0:]], axis=0)

# Print: training data frame
x_training_df

In [ ]:
# Input layers
one_hot_input = keras.Input(shape=(3,), name='one_hot_input')
winner_input = keras.Input(shape=(1,), name="winner_input")

# Define machine learning model
inputs = keras.layers.Concatenate()([one_hot_input, winner_input])
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(3, activation="softmax", name="predictions")(x)

In [ ]:
# Create machine learning model
model = keras.Model(inputs=[one_hot_input, winner_input], outputs=outputs)

In [ ]:
# Compile machine learning model
model.compile(loss='categorical_crossentropy',
          optimizer=keras.optimizers.Adam(),
          metrics=['accuracy'])

In [ ]:
# Plot machine learning model
keras.utils.plot_model(model, "RPS.png", show_shapes=True)

In [ ]:
# Prepare current training data tensors -> T(n)
x_train = x_training_df.iloc[0:,1:]
x_train_one_hot = tf.constant(np.array(x_train['H'].tolist()))
x_train_winner = tf.constant(x_train['W'].astype('int32'))

In [ ]:
# Prepare following training data tensors -> T(n+1)
y_train = y_training_df.iloc[0:,1:2]
y_train = tf.constant(np.array(y_train['H'].tolist()))
y_train

In [ ]:
# Prepare current validation data tensors -> V(n)
x_val = x_validation_df.iloc[0:,1:]
x_val_one_hot = tf.constant(np.array(x_val['H'].tolist()))
x_val_winner = tf.constant(x_val['W'].astype('int32'))

In [ ]:
# Prepare following validation data tensors -> V(n+1)
y_val = y_validation_df.iloc[0:,1:2]
y_val = tf.constant(np.array(y_val['H'].tolist()))
y_val

In [ ]:
# Train the model
print("Fit model on training data")
history = model.fit(
    {'one_hot_input': x_train_one_hot, 'winner_input': x_train_winner},    # letztes T, letzter Siegwert, T(n) + W(n)
    y_train,    # neues T, T(n+1)
    batch_size=32,
    epochs=10,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=({'one_hot_input': x_val_one_hot, 'winner_input': x_val_winner}, y_val),
)

In [ ]:
history.history

In [ ]:
# Save the model
model.save("./rps_model.keras", overwrite=True)